# Building a RAG System with LangChain & ChromaDB

## Introduction
Retrieval-Augmented Generation(RAG) is a powerful technique that combines the capabilities of large language models with external knowledge retrieval. This notebook will walk you through building complete RAG System using:

- LangChain: A framework for developing applications powered by language models 
- ChromaDB: An open-source vectorDB for storing & retrieving embeddings
- LLM: For embeddings & language model (you can use any model)

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
## langchain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document

#Vector Stores
from langchain_community.vectorstores import Chroma

##  utility imports
import numpy as np
from typing import List

In [9]:
# RAG Architecture Overview
print("""
RAG (Retrieval-Augmented Generation) Architecture:

1. Document Loading: Load documents from various sources
2. Document Splitting: Break documents into smaller chunks
3. Embedding Generation: Convert chunks into vector representations
4. Vector Storage: Store embeddings in ChromaDB
5. Query Processing: Convert user query to embedding
6. Similarity Search: Find relevant chunks from vector store
7. Context Augmentation: Combine retrieved chunks with query
8. Response Generation: LLM generates answer using context

Benefits of RAG:
- Reduces hallucinations
- Provides up-to-date information
- Allows citing sources
- Works with domain-specific knowledge
""")


RAG (Retrieval-Augmented Generation) Architecture:

1. Document Loading: Load documents from various sources
2. Document Splitting: Break documents into smaller chunks
3. Embedding Generation: Convert chunks into vector representations
4. Vector Storage: Store embeddings in ChromaDB
5. Query Processing: Convert user query to embedding
6. Similarity Search: Find relevant chunks from vector store
7. Context Augmentation: Combine retrieved chunks with query
8. Response Generation: LLM generates answer using context

Benefits of RAG:
- Reduces hallucinations
- Provides up-to-date information
- Allows citing sources
- Works with domain-specific knowledge



# 1. Sample Data

In [10]:
## create sample documents
sample_docs = [
    """
    Machine Learning Fundamentals
    
    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties.
    """,
    
    """
    Deep Learning and Neural Networks
    
    Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of interconnected 
    nodes. Deep learning has revolutionized fields like computer vision, natural language 
    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly 
    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers 
    excel at sequential data processing.
    """,
    
    """
    Natural Language Processing (NLP)
    
    NLP is a field of AI that focuses on the interaction between computers and human language. 
    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, 
    machine translation, and question answering. Modern NLP heavily relies on transformer 
    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand 
    context and relationships between words in text.
    """
]

sample_docs

['\n    Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.\n    ',
 '\n    Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective f

In [11]:
## save sample documents to files
os.makedirs("data/", exist_ok=True)
for i,doc in enumerate(sample_docs):
     with open(f"data/doc_{i}.txt", 'w') as f:
        f.write(doc)
print(f"sample doc create in: data")

sample doc create in: data


2. Document Loading

In [12]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

#Load Documents from directory
loader=DirectoryLoader(
    "data",
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={'encoding': 'utf-8'}
)
documents=loader.load()
print(f"Loader {len(documents)} documents")
print(documents[0].page_content[:100]+"...")
print(f"\n\n")
print(documents[0])

Loader 3 documents

    Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that...



page_content='
    Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties.
    ' metadata={'source': 'data\\doc_0.txt'}


# Document Splitting

In [13]:
# Initialise text splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, # Maximum size of each chunk
    chunk_overlap=50, # overlap between the chunks
    length_function=len,
    separators=[" "] #Heirarchy of separators
)
chunks=text_splitter.split_documents(documents)

print(f"Created {len(chunks)} chunks from {len(documents)} documents")
print(f"\nchunk example: ")
print(f"\n\n")
print(chunks[0])

Created 5 chunks from 3 documents

chunk example: 



page_content='Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through' metadata={'source': 'data\\doc_0.txt'}


In [14]:
chunks

[Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': 'data\\doc_0.txt'}, page_content='data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of in

# Embedding Models

In [15]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
embedding_model

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
sample_text="Machine learnign is fascinating"
query_embeddings=embedding_model.embed_query(sample_text)
query_embeddings


[-0.08156096935272217,
 -0.068397656083107,
 0.01674613170325756,
 -0.012387065216898918,
 0.028286781162023544,
 -0.0300765261054039,
 -0.001563390949741006,
 -0.009739561937749386,
 -0.04644002765417099,
 0.0019273207290098071,
 -0.04317094758152962,
 0.11336471140384674,
 0.008018924854695797,
 -0.0622156523168087,
 -0.06404369324445724,
 -0.002163096098229289,
 -0.021521706134080887,
 0.011608379893004894,
 -0.015610083006322384,
 -0.08683598041534424,
 -0.03697584941983223,
 -0.006414297502487898,
 0.023288149386644363,
 0.009337526746094227,
 0.014802733436226845,
 0.03275873512029648,
 0.05226026847958565,
 0.05248754471540451,
 0.11069823801517487,
 -0.08441156148910522,
 -0.0023191983345896006,
 0.09471874684095383,
 -0.013603033497929573,
 -0.0034698965027928352,
 -0.05242239311337471,
 0.02582346647977829,
 0.05688219144940376,
 -0.019815709441900253,
 0.0009714356274344027,
 -0.04124041274189949,
 -0.0793374702334404,
 -0.03534483537077904,
 0.023837057873606682,
 0.0100408

In [17]:
print(len(query_embeddings))

384


# Initialize the chromaDB vector store & STores hthe chunks in vector representation

In [20]:
from chromadb import Client
from chromadb.config import Settings

# Remove-Item -Recurse -Force chroma_db

In [22]:
# Create a ChromaDB vector store
persistent_directory="./chroma_db"

##Initialise the chromaDB with Hugging Face embeddings
vector_store=Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory=persistent_directory,
    collection_name="rag_collection"
)
# vector_store=Chroma.from_documents(
#     Settings(
#         persist_directory=persistent_directory,
#         anonymized_telemetry=False
#     )
# )
print(f"length of chunks: {len(chunks)}")
print(f"Vector store created with {vector_store._collection.count()} vectors")
print(f"Persisted to: {persistent_directory}")

length of chunks: 5
Vector store created with 45 vectors
Persisted to: ./chroma_db


In [24]:
# vector_store_ = Chroma(
#     collection_name="rag_collection",
#     persist_directory="./chroma_db",
#     embedding_function=embedding_model,
#     client_settings=Settings(
#         persist_directory="./chroma_db",
#         anonymized_telemetry=False
#     )
# )
# vector_store._collection.delete(where={})
# vector_store.persist()

# Text Similarity Search

In [23]:
query="What are the types of machine learning"

similar_docs=vector_store.similarity_search(query, k=3)
similar_docs

[Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, w

In [25]:
query_2="What is NLP"

similar_docs=vector_store.similarity_search(query_2, 3)
similar_docs

[Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Natural Language Processing (NLP)\n\n    NLP is a field of AI that focuses on the interaction between computers and human language. \n    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, \n    machine translation, and question answering. Modern NLP heavily relies on transformer \n    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand \n    context and relationships between words in text.'),
 Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Natural Language Processing (NLP)\n\n    NLP is a field of AI that focuses on the interaction between computers and human language. \n    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, \n    machine translation, and question answering. Modern NLP heavily relies on transformer \n    architectures like BERT, GPT, and T5. These models use attention mec

In [26]:
query3="What is Deep Learning"
similar_docs=vector_store.similarity_search(query3,3)
similar_docs

[Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neura

In [27]:
print(f"Query: {query3}")
print(f"\nTop {len(similar_docs)} similar chunks:")
for i, doc in enumerate(similar_docs):
    print(f"\n--- Chunk {i+1} ---")
    print(doc.page_content[:200] + "...")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")

Query: What is Deep Learning

Top 3 similar chunks:

--- Chunk 1 ---
Deep Learning and Neural Networks

    Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of i...
Source: data\doc_1.txt

--- Chunk 2 ---
Deep Learning and Neural Networks

    Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of i...
Source: data\doc_1.txt

--- Chunk 3 ---
Deep Learning and Neural Networks

    Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of i...
Source: data\doc_1.txt


# Advanced Similarity Search

In [28]:
results_scores=vector_store.similarity_search_with_score(query3, k=3)
results_scores

[(Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers'),
  0.5244521498680115),
 (Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recogni

#### Understanding Similarity Scores
The similarity score represents how closely related a document chunk is to your query. The scoring depends on the distance metric used:

ChromaDB default: Uses L2 distance (Euclidean distance)

- Lower scores = MORE similar (closer in vector space)
- Score of 0 = identical vectors
- Typical range: 0 to 2 (but can be higher)


Cosine similarity (if configured):

- Higher scores = MORE similar
- Range: -1 to 1 (1 being identical)

### Initialize LLM, RAG Chain, Prompt Template, Query the RAG System

In [29]:
from langchain.chat_models import init_chat_model
import os
from dotenv import load_dotenv

load_dotenv()

openAIModel=("openai/gpt-oss-120b")
llm_model=init_chat_model(
                        model=openAIModel,
                        model_provider="OPENAI",
                        api_key=os.getenv("GROQ_API_KEY"),
                        base_url="https://api.groq.com/openai/v1",
                        temperature=0.3
)

response=llm_model.invoke("Explain yourself ?")
print(response.content)


I’m ChatGPT, a conversational AI created by OpenAI. Here’s a quick rundown of what I am and how I work:

| Aspect | What it means |
|--------|----------------|
| **Core technology** | I’m built on the GPT‑4 family of large language models. Think of me as a very big statistical pattern‑matcher that has read billions of words of text from books, articles, code, forums, and many other sources. |
| **Training** | During training I was exposed to a wide variety of topics—science, history, art, programming, everyday life, and more—so I can generate text that is coherent, context‑aware, and often useful. I never “remember” specific documents, but I learn the statistical relationships between words and concepts. |
| **Knowledge cutoff** | My training data goes up to **September 2021**, with a small amount of additional fine‑tuning data up to **June 2024**. Anything that happened after that (e.g., the latest news, brand‑new research, or recent product releases) may be unknown to me. |
| **How I

### Modern RAG Chain

In [30]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [32]:
## Convert vector store to retriever
retriever=vector_store.as_retriever(
    search_kwargs={"k":3} ## Retrieve top 3 relevant chunks
)
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001AD1FED2E90>, search_kwargs={'k': 3})

In [35]:
## Create a prompt template
from langchain_core.prompts import ChatPromptTemplate
system_prompt="""
"You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Context: {context}
"""

prompt=ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])


In [36]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n"You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don\'t know the answer, just say that you don\'t know. \nUse three sentences maximum and keep the answer concise.\n\nContext: {context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

### What is create_stuff_documents_chain ?
Create_stuff_documents_chain creates a chain that "stuffs" (inserts) all retrieved documents into a single prompt & sends it to the LLM. it's called "stuff" --> as it really stuffs all the documents in to context window at once

In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain

doc_chain=create_stuff_documents_chain(llm_model, prompt)
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n"You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don\'t know the answer, just say that you don\'t know. \nUse three sentences maximum and keep the answer concise.\n\nContext: {context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001AD1FED3C50>, async_client=<ope

#### This Chain:
- Takes retrieved documents
- "Stuffs" them into the prompt's {context} placeholder
- Sends the complete prompt to LLM
- Returns LLMs response 

Create_Stuff_documents  

    -- takes retrieved docs as i/p 
    --> sends this docs to LLM
    --> Returns LLMs response

Retriever  
-- Process the documents  
--> Retrieves them  
--> stuffs the whole retrieved+context to LLMs  
--> returns LLMs response  

In [ ]:
create_retrieval_chain

In [44]:
vector_store

In [45]:
chunks

[Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': 'data\\doc_0.txt'}, page_content='data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of in

#### Advanced RAG Techniques - Conversational Memory

- Maintaining context across multiple interactions
- Follow-up questions that reference previous answers pronoun resolution, ex: "it", "they", "that" 
- Natural_Dialogue_Flow: users dont repeat context

##### PAIN-POINT to be solved:
- Traditional RAG retrieves docs based on curr query, missing imp context from conversation

#### Solution: 
- Query Reformulation: Transform context-dependent questions into standalone queries -- Context-Aware Retrieval 
- "create_history_aware_retrieval" : retriever understands the context
- Messages/Placeholder: Placeholder for chat history in prompts 
- HumanMessage / AIMessage

In [47]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage,  AIMessage

In [48]:
## Create a prompt that includes the chat history
contexualize_q_sys_pro="""Given a chat history and the latest user question 
which might reference context in the chat history, formulate a standalone question 
which can be understood without the chat history. Do NOT answer the question, 
just reformulate it if needed and otherwise return it as is."""

contexualize_q_prompt=ChatPromptTemplate.from_messages([
    ("system", contexualize_q_sys_pro),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

In [51]:
## create history aware retriever
history_aware_retriever=create_history_aware_retriever(
    llm_model, #LLM
    retriever, #Retriever
    contexualize_q_prompt #Contexual System Prompt
)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001AD1FED2E90>, search_kwargs={'k': 3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, T

In [55]:
# Create a new document chain with history

qa_sys_prompt="""You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Context: {context}"""

qa_prompt=ChatPromptTemplate.from_messages([
    ("system", qa_sys_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

docStuff=create_stuff_documents_chain(
    llm_model,
    qa_prompt
)

conversational_rag_chain=create_retrieval_chain(
    history_aware_retriever,
    docStuff
)

print("Conversational RAG chain created!")

Conversational RAG chain created!


In [56]:
chat_history=[]
result1=conversational_rag_chain.invoke(
    {
        "chat_history": chat_history,
        "input": "What is machine learning"
    }
)
print(f"Q: what is machine learning")
print(f"A: {result1['answer']}")

Q: what is machine learning
A: Machine learning is a subset of artificial intelligence that lets systems learn from data and improve their performance without being explicitly programmed. It encompasses methods that enable computers to recognize patterns, make predictions, or take actions based on experience. The main approaches include supervised, unsupervised, and reinforcement learning.


In [57]:
chat_history.extend([
    HumanMessage(content="What is machine learning"),
    AIMessage(content=result1['answer'])
])

In [58]:
chat_history

[HumanMessage(content='What is machine learning', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Machine learning is a subset of artificial intelligence that lets systems learn from data and improve their performance without being explicitly programmed. It encompasses methods that enable computers to recognize patterns, make predictions, or take actions based on experience. The main approaches include supervised, unsupervised, and reinforcement learning.', additional_kwargs={}, response_metadata={})]

In [60]:
## Follow-up Question
result2=conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "what are its main types"
})
result2

{'chat_history': [HumanMessage(content='What is machine learning', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Machine learning is a subset of artificial intelligence that lets systems learn from data and improve their performance without being explicitly programmed. It encompasses methods that enable computers to recognize patterns, make predictions, or take actions based on experience. The main approaches include supervised, unsupervised, and reinforcement learning.', additional_kwargs={}, response_metadata={})],
 'input': 'what are its main types',
 'context': [Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learnin

In [61]:
result2['answer']

'The main types of machine learning are\u202f**supervised learning**, which trains models on labeled data;\u202f**unsupervised learning**, which discovers patterns in unlabeled data; and\u202f**reinforcement learning**, where an agent learns by receiving feedback from its actions in an environment.'